# Lecture 26. Incremental Data Ingestion from Files

### Reference
  - [Documentation > Ingest data into a Databricks lakehouse > Ingest data from cloud object storage > What is Auto Loader?](https://docs.databricks.com/ingestion/auto-loader/index.html)  



## What is Incremental Data Ingestion?

By ***incremental data ingestion***, we mean the ability to *load data from new files that have been encountered since the last ingestion*. 

So each time we run our data pipeline, we don't need to reprocess the files we have processed before. We need to process only the new arriving data files.  (*Reduces redundant processing*)

## Methods for Incremental Data Ingestion

Databricks provides two mechanisms for incrementally and efficiently processing new data files as they arrive in a storage location, which are `COPY INTO` SQL command and Auto Loader.

### `COPY INTO` Command

`COPY INTO` is a **SQL command** that allows users to load data from a file location into a Delta table. It loads data *idempotently* and *incrementally*. It means each time you run the command, it will **load only the new files** from the source location, while *the files that have been loaded before are simply skipped*.

#### Format

The command is pretty simple: 

```sql
COPY INTO my_table
FROM '/path/to/files’
FILEFORMAT = <format>
FORMAT_OPTIONS (<format options>)
COPY_OPTIONS (<copy options>);
```

- `COPY INTO <target_table> FROM <source_location>`
- And we specify the format of the source file to load, for example, `CSV` or `parquet`, 
- and any related format options, 
- in addition to any option to control the operation of the `COPY INTO` command.

#### Example

Here, for example, we are loading from *CSV files*, having a header and a specific delimiter. And in the `COPY OPTIONS`, we are specifying that the schema can evolve according to the incoming data.

```sql
COPY INTO my_table
FROM '/path/to/files'
FILEFORMAT = CSV
FORMAT_OPTIONS ('delimiter' = '|',
                'header' = 'true')
COPY_OPTIONS ('mergeSchema' = 'true')
```

### Auto Loader

The second method to load data incrementally from files is ***Auto Loader***, which uses *structured streaming* in Spark to efficiently process new data files as they arrive in a storage location. 
You can use Auto Loader to *load billions of files into a table*.
Auto Loader can scale to support near real-time ingestion of millions of files per hour. 

#### Auto loader Checkpointing

And since it is based on *Spark Structured Streaming*, Auto Loader uses *checkpointing* to track the ingestion process and to store metadata of the discovered files. 

So Auto Loader ensures that data files are processed *exactly once*. 

And in case of failure, Auto Loader can resume from where it left off. (Fault tolerance)

#### Auto Loader in PySpark API

To work with Auto Loader, we use the `readStream` and `writeStream` methods. 

```python
spark.readStream
        .format("cloudFiles")
        .option("cloudFiles.format", <source_format>)
        .load('/path/to/files’)
     .writeStream
        .option("checkpointLocation", <checkpoint_directory>)
        .table(<table_name>)
```

Auto Loader has a specific format for *Stream Reader* called `cloudFiles`. 

And in order to specify the format of the source files, we simply use the `cloudFiles.format` option. 

And with the `load` function, we provide the location of the source files.
In this location, Auto Loader will detect new files as they arrive and queue them for ingestion. 

Once we read the files, we write the data into a target table using the sream writer, where we provide the location to store the checkpointing information.

#### Auto Loader + Schema

Auto Loader can automatically infer the schema of your data. 
It can detect any update to the fields of the source dataset. 

However, to avoid this *inference cost* at every startup of your stream, the inferred schema can be stored in a location to be used later.

```python
spark.readStream
        .format("cloudFiles")
        .option("cloudFiles.format", <source_format>)
        .option("cloudFiles.schemaLocation", <schema_directory>)
        .load('/path/to/files’)
     .writeStream
        .option("checkpointLocation", <checkpoint_directory>)
        .option("mergeSchema", “true”)
        .table(<table_name>)
```

To provide the location where Auto Loader can store the schema, use the option: `cloudFiles.schemaLocation`

And this location could be simply the same as the checkpoint location.


## When to Use Auto Loader vs. `COPY INTO`

Now we know everything about Auto Loader. The question is, when to use Auto Loader and when to use `COPY INTO` command?

Here are important things to consider when choosing between `COPY INTO` and Auto Loader:

- If you are going to ingest files in the order of *thousands*, you can use the `COPY INTO` command.
- However, if you are expecting files in the order of *millions* or more over time, use Auto Loader.

In addition, Auto Loader can split the processing into multiple batches, so it is more efficient at scale. Databricks recommends using Auto Loader as a general best practice when ingesting data from *cloud object storage*.



## Conclusion

Let us now switch to the Databricks platform to [work with Auto Loader](./Lecture-27__Auto-Loader-(Hands-On).ipynb).
